<a href="https://colab.research.google.com/github/kanjitp/CS4225/blob/main/cs4225_a2_colab_KanJitpakdi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Task 1: Spark SQL (15m)

In [3]:
# Setup Spark
# ===============
# Installing Spark needs to be done once each time you re-open this notebook. It should take around 10-30 seconds.
# ===============
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

# unzip the spark file to the current folder
!tar xf spark-3.3.2-bin-hadoop3.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

# install findspark using pip
!pip install -q findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
# After downloading dataset, you should have the files in your Files (click the folder icon in the left sidebar)
!wget -O Products_table.csv https://drive.google.com/uc?id=1FG0rGWSPWALcmFo3feHUF5TK5AP7mMwH&export=download #products
!wget -O Sales_table.csv https://drive.google.com/uc?id=1l1fr_s67JjGGsXt3fIz_769pKPZg-jhU&export=download #sales
!wget -O Sellers_table.csv https://drive.google.com/uc?id=1YTTYU5Cwgvau1Z7b1ShmcIhrO3VN-Zhq&export=download #sellers

--2023-04-01 07:44:20--  https://drive.google.com/uc?id=1FG0rGWSPWALcmFo3feHUF5TK5AP7mMwH
Resolving drive.google.com (drive.google.com)... 172.253.62.102, 172.253.62.101, 172.253.62.139, ...
Connecting to drive.google.com (drive.google.com)|172.253.62.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-a0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ab6p3ngbtar5an2on9cmgujhd9g8fv8s/1680335025000/06948221057362969045/*/1FG0rGWSPWALcmFo3feHUF5TK5AP7mMwH?uuid=f06e982f-d973-436c-9393-57d6a308ac28 [following]
--2023-04-01 07:44:22--  https://doc-0g-a0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ab6p3ngbtar5an2on9cmgujhd9g8fv8s/1680335025000/06948221057362969045/*/1FG0rGWSPWALcmFo3feHUF5TK5AP7mMwH?uuid=f06e982f-d973-436c-9393-57d6a308ac28
Resolving doc-0g-a0-docs.googleusercontent.com (doc-0g-a0-docs.googleusercontent.com)... 142.251.16.132, 2607:f8b0:4004:c17::84
Connecting to doc-0g

In [5]:
# read csv files into dataframes, you can work with the 3 tables after running this code
products_table = spark.read.option('header', True).option('inferSchema', True).csv("/content/Products_table.csv").repartition(1).cache()
sales_table = spark.read.option('header', True).option('inferSchema', True).csv("/content/Sales_table.csv").repartition(1).cache()
sellers_table = spark.read.option('header', True).option('inferSchema', True).csv("/content/Sellers_table.csv").repartition(1).cache()

In [12]:
products_table

DataFrame[product_id: int, product_name: string, price: int]

In [13]:
sales_table

DataFrame[order_id: int, product_id: int, seller_id: int, num_of_items_sold: int]

In [14]:
sellers_table

DataFrame[seller_id: int, seller_name: string, rating: int]

In [6]:
# add all imports
from pyspark.sql.functions import sum, asc, desc, col

In [64]:
# (a) Output the top 3 most popular products sold among all sellers [2m]
# Your table should have 1 column(s): [product_name]

top_3_products = (
    sales_table
    .groupBy('product_id')
    .sum('num_of_items_sold')
    .withColumnRenamed('sum(num_of_items_sold)', 'total_items_sold')
    .join(products_table, 'product_id')
    .select('product_name')
    .orderBy(col('total_items_sold').desc())
    .limit(3)
)

top_3_products.show()

+-------------+
| product_name|
+-------------+
|product_51270|
|product_18759|
|product_59652|
+-------------+



In [63]:
# (b) Find out the total sales of the products sold by sellers 1 to 10 and output the top most sold product [2m]
# Your table should have 1 column(s): [product_name]


from pyspark.sql.functions import col

# filter the sellers for seller 1 to 10
filtered_sellers = sales_table.filter(col('seller_id').between(1, 10))

top_product_for_seller_1_to_10 = (
    filtered_sellers
    .groupBy('product_id')
    .agg(sum('num_of_items_sold').alias('total_items_sold'))
    .join(products_table, 'product_id')
    .select('product_name')
    .orderBy(desc('total_items_sold'))
    .limit(1)
)

top_product_for_seller_1_to_10.show()

+-------------+
| product_name|
+-------------+
|product_36658|
+-------------+



In [62]:
# (c) Compute the combined revenue earned from sellers where seller_id ranges from 1 to 500 inclusive. [3m]
# Your table should have 1 column(s): [total_revenue]

revenue = (
    sales_table
    .filter(col('seller_id').between(1, 500))
    .join(products_table, 'product_id')
    .withColumn('revenue', col('num_of_items_sold') * col('price'))
    .agg(sum('revenue').alias('total_revenue'))
)

revenue.show()

+-------------+
|total_revenue|
+-------------+
|    160916699|
+-------------+



In [66]:
# (d) Among sellers with rating >= 4 who have achieved a combined number of products sold >= 3000, find out the top 10 most expensive product sold by any of the sellers. (If there are multiple products at the same price, please sort them in ascending order of product_id) [8m]
# Your table should have 1 column(s): [product_name]
# To get the full mark, your query should not run for more than 1 min

high_rating_sellers = sellers_table.filter(col('rating') >= 4)

# sellers with rating >= 4 who have achieved a combined number of products sold >= 3000
sales_high_rating_sellers = (
    sales_table
    .join(high_rating_sellers, 'seller_id')
    .groupBy('seller_id')
    .agg(sum('num_of_items_sold').alias('total_items_sold'))
    .filter(col('total_items_sold') >= 3000)
)

# For video illustration
sales_high_rating_sellers.show(10)

# sellers and their product that they sold
seller_with_product_sold = (
    sales_high_rating_sellers
    .join(sales_table, 'seller_id')
)

# For video illustration
seller_with_product_sold.show(10)

# top 10 most expensive product sold by any of the sellers.
# If there are multiple products at the same price, please sort them in ascending order of product_id)
top_expensive_products = (
    seller_with_product_sold
    .join(products_table, 'product_id')
    .select('product_name')
    .orderBy(desc('price'), asc('product_id'))
    .limit(10)
)

# Final Result
top_expensive_products.show(10)


+---------+----------------+
|seller_id|total_items_sold|
+---------+----------------+
|    23608|            6285|
|    34610|            4441|
|     1543|            4147|
|    10269|            3071|
|    38703|            3897|
|    35872|            4955|
|    39620|            5147|
|    31478|            3384|
|    16139|            3577|
|    13309|            4448|
+---------+----------------+
only showing top 10 rows

+---------+----------------+--------+----------+-----------------+
|seller_id|total_items_sold|order_id|product_id|num_of_items_sold|
+---------+----------------+--------+----------+-----------------+
|    23608|            6285|  293756|     95179|              673|
|    23608|            6285|  175321|     51037|              531|
|    23608|            6285|  149971|     49213|              840|
|    23608|            6285|  147765|     16749|              898|
|    23608|            6285|  128426|     51717|               39|
|    23608|            6285|   6

## Task 2: Spark ML (10m)

In [ ]:
# Setup Spark
# ===============
# Installing Spark needs to be done once each time you re-open this notebook. It should take around 10-30 seconds.
# ===============
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

# unzip the spark file to the current folder
!tar xf spark-3.3.2-bin-hadoop3.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

# install findspark using pip
!pip install -q findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
# After downloading dataset, you should have the files in your Files (click the folder icon in the left sidebar)
!wget -O bank_train.csv https://drive.google.com/uc?id=1kEP94BfULB3gUMl_IQCg9wuX4IJRajMC&export=download #products
!wget -O bank_test.csv https://drive.google.com/uc?id=1EqX4liL5iWbwqyJ_lFaYvYZvgBFwpwSJ&export=download #bank_test

--2023-03-05 13:59:46--  https://drive.google.com/uc?id=1kEP94BfULB3gUMl_IQCg9wuX4IJRajMC
Resolving drive.google.com (drive.google.com)... 142.251.8.102, 142.251.8.113, 142.251.8.100, ...
Connecting to drive.google.com (drive.google.com)|142.251.8.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-a8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/6a3e19tqodgib3pccobtlthqmdiclhtv/1678024725000/08487103376102314083/*/1kEP94BfULB3gUMl_IQCg9wuX4IJRajMC?uuid=d11faedd-8c48-4734-aa0b-18efe2a56d2e [following]
--2023-03-05 13:59:47--  https://doc-0c-a8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/6a3e19tqodgib3pccobtlthqmdiclhtv/1678024725000/08487103376102314083/*/1kEP94BfULB3gUMl_IQCg9wuX4IJRajMC?uuid=d11faedd-8c48-4734-aa0b-18efe2a56d2e
Resolving doc-0c-a8-docs.googleusercontent.com (doc-0c-a8-docs.googleusercontent.com)... 142.250.157.132, 2404:6800:4008:c13::84
Connecting to doc-0c-a8

In [ ]:
bank_train_location = "/content/bank_train.csv"
bank_test_location = "/content/bank_test.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bank_train = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(bank_train_location)

bank_test = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(bank_test_location)

Build ML model to predict whether the customer will subscribe bank deposit service or not. Train the model using training set and evaluate the model performance (e.g. accuracy) using testing set.


*   You can explore different methods to pre-process the data and select proper features
*   You can explore different methods to pre-process the data and select proper features
*   Present the final testing accuracy.

In [ ]:
# data preparation (4m)


In [ ]:
# model building (4m)

In [ ]:
# model evaluation (2m)